In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("cleaned_date.csv")

In [4]:
#  generating dummies
df_d=pd.get_dummies(df)
# extracting train with 'price' and test without 'price'
df_train=df_d.dropna() # dropping all null values which are only present in target column 'price'
df_test=df_d[df_d['Price'].isnull()]
df_test.drop(['Price'],axis=1,inplace=True)


C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
df_train.shape

(10683, 175)

In [8]:
x_train=df_train.drop(['Price'],axis=1)

y_train=df_train['Price']
x_test=pd.get_dummies(df_test)

In [9]:
print(x_train.shape)
print(x_test.shape)

(10683, 174)
(2671, 174)


In [10]:
#feature selection using backward elimination
import statsmodels.formula.api as sma
import statsmodels.api as sm

f_train = x_train
f_test = x_test
Price = y_train
features = list(x_train.columns)
explain = features.copy()


import scipy.stats as st
from statsmodels.tools import add_constant as add_constant
f_train['constant']=1
#backward selection
# Automating the selection of optimal features using backward elimination, here probability threshold is 0.05.
while(True):
    model = sm.OLS(endog = Price, exog = f_train[explain]).fit()
    pvals = model.pvalues
    max_pval = pvals[(pvals == pvals.max()) & (pvals > 0.05)]
    if(max_pval.empty == False):
        explain.remove(max_pval.index[0])
        continue
    else:
        break
        
explain

['Total_Stops',
 'Arrival_min',
 'Date_of_Journey_month',
 'Date_of_Journey_day',
 'Additional_Info_1 Long layover',
 'Additional_Info_1 Short layover',
 'Additional_Info_2 Long layover',
 'Additional_Info_Business class',
 'Additional_Info_Change airports',
 'Additional_Info_In-flight meal not included',
 'Additional_Info_No check-in baggage included',
 'Additional_Info_No info',
 'Airline_Air Asia',
 'Airline_GoAir',
 'Airline_IndiGo',
 'Airline_Jet Airways',
 'Airline_Jet Airways Business',
 'Airline_Multiple carriers',
 'Airline_SpiceJet',
 'Airline_Trujet',
 'Destination_Banglore',
 'Destination_Cochin',
 'Destination_Delhi',
 'Destination_Hyderabad',
 'Destination_Kolkata',
 'Destination_New Delhi',
 'Route_BLR → BDQ → DEL',
 'Route_BLR → BOM → AMD → DEL',
 'Route_BLR → BOM → BHO → DEL',
 'Route_BLR → BOM → DEL',
 'Route_BLR → BOM → IDR → DEL',
 'Route_BLR → BOM → JDH → DEL',
 'Route_BLR → BOM → NAG → DEL',
 'Route_BLR → CCU → BBI → DEL',
 'Route_BLR → GAU → DEL',
 'Route_BLR → H

In [11]:
len(explain)


91

In [13]:

from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
LR=LinearRegression()

LR_bag=BaggingRegressor(base_estimator=LR,n_estimators=25,random_state=0)
DT_reg=DecisionTreeRegressor(max_depth=4,criterion='entropy',random_state=0)
DT_bag=BaggingRegressor(n_estimators=10,random_state=0)
RF=RandomForestRegressor(n_estimators=130,criterion='entropy',random_state=0)
#Boosting models

RF_boost=AdaBoostRegressor(base_estimator=RF,n_estimators=100)
GB_boost=GradientBoostingRegressor(n_estimators=100)


In [16]:
from sklearn.preprocessing import StandardScaler


In [17]:
rff=RandomForestRegressor(bootstrap=False, max_features=0.4, min_samples_leaf=4, min_samples_split=9, n_estimators=100,random_state=1)

In [20]:
# dropping constant
x_train.drop(['constant'],axis=1,inplace=True)

In [21]:
pred=rff.fit(x_train,y_train).predict(x_test)

dg=pd.DataFrame(x_train)
# feature importance
a=pd.DataFrame({'features':dg.columns,'importance':rff.feature_importances_})
a.sort_values(by=['importance'],ascending=False)



,features,importance
8,Duration,0.278612
1,Total_Stops,0.143594
7,Date_of_Journey_day,0.093017
23,Airline_Jet Airways,0.083005
6,Date_of_Journey_month,0.044443
14,Additional_Info_In-flight meal not included,0.042506
22,Airline_IndiGo,0.035497
17,Additional_Info_No info,0.035170
24,Airline_Jet Airways Business,0.033096
42,Route_BLR → BOM → DEL,0.018670


In [22]:
feature=a[a['importance']!=0]

f=feature.features

b=list(f.values)


a[a['importance']==0]['features'].count()


54

In [23]:

g=list(feature['features'])
g

['Unnamed: 0',
 'Total_Stops',
 'Arrival_hour',
 'Arrival_min',
 'Dep_hour',
 'Dep_min',
 'Date_of_Journey_month',
 'Date_of_Journey_day',
 'Duration',
 'Additional_Info_1 Long layover',
 'Additional_Info_Business class',
 'Additional_Info_Change airports',
 'Additional_Info_In-flight meal not included',
 'Additional_Info_No check-in baggage included',
 'Additional_Info_No info',
 'Airline_Air Asia',
 'Airline_Air India',
 'Airline_GoAir',
 'Airline_IndiGo',
 'Airline_Jet Airways',
 'Airline_Jet Airways Business',
 'Airline_Multiple carriers',
 'Airline_Multiple carriers Premium economy',
 'Airline_SpiceJet',
 'Airline_Vistara',
 'Destination_Banglore',
 'Destination_Cochin',
 'Destination_Delhi',
 'Destination_Hyderabad',
 'Destination_Kolkata',
 'Destination_New Delhi',
 'Route_BLR → AMD → DEL',
 'Route_BLR → BBI → DEL',
 'Route_BLR → BDQ → DEL',
 'Route_BLR → BOM → AMD → DEL',
 'Route_BLR → BOM → BHO → DEL',
 'Route_BLR → BOM → DEL',
 'Route_BLR → BOM → IDR → DEL',
 'Route_BLR → BOM

In [24]:
# new model after removing columns with 0 feature importance
xtrain=x_train.loc[:,g]
xtest=x_test.loc[:,g]

In [25]:
rf_2=RandomForestRegressor(bootstrap=False, max_features=0.4, min_samples_leaf=4, min_samples_split=9, n_estimators=100,random_state=1)

In [26]:
y_predd=rf_2.fit(xtrain,y_train).predict(xtest)

In [27]:
rf_2.score(xtrain,y_train)

0.9564401793371853

In [ ]:
model1=pd.DataFrame(y_predd)
model1.to_csv('submission1.csv')